In [1]:
from data_processing import prepare_dataset, assign_colors
from align_func import execute_global_registration, icp_registration, colored_icp
from GA import evaluation_colored_icp, mutation, generator
from deap import algorithms, base, creator, tools
from datetime import date

import open3d as o3d
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob 
import copy
import random
import os
import sys



# Change variables here

In [8]:
### File directories ###
source_position_name = "./datasets/xyz_source.xls"
source_color_name = "./datasets/sox2_source.xls"
target_position_name = "./datasets/xyz_target.xls"
target_color_name= "./datasets/sox2_target.xls"

### DEAP runs ### 

image_name = "sox2"
save_results_path = "./results"
results_name = "NSGA3_coloredICP_run"
num_deap_runs = 20


# Importing files and data processing 

In [10]:
## Source ##
source_position_excel = [pd.read_excel(source_position_name, skiprows = [0,1,2], header = 0, usecols = [0,1,2])]
source_color_excel = [pd.read_excel(source_color_name, skiprows = [0,1,2], header = 0, usecols = [0])]

## Target ## 
target_position_excel = [pd.read_excel(target_position_name, skiprows = [0,1,2], header = 0, usecols = [0,1,2])]
target_color_excel = [pd.read_excel(target_color_name, skiprows = [0,1,2], header = 0, usecols = [0])]

positions_excel_list = source_position_excel + target_position_excel
positions_names_list = [source_position_name] + [target_position_name]
color_excel_list = source_color_excel + target_color_excel

source_color =color_excel_list[0].to_numpy(dtype='float64')
target_color =color_excel_list[1].to_numpy(dtype='float64')

source, target, source_processed, target_processed, source_fpfh, target_fpfh = prepare_dataset(positions_excel_list, positions_names_list)
assign_colors(source,target,color_excel_list,"viridis")

:: Point Cloud was not downsampled
:: Estimate normal with search radius 20.
:: Compute FPFH feature with search radius 50.
---------------------------------------
:: Point Cloud was not downsampled
:: Estimate normal with search radius 20.
:: Compute FPFH feature with search radius 50.
---------------------------------------
:: Assigned colors to point clouds


# DEAP initialisations

In [11]:
# results.fitness (MAX), results.inlier_mse (MIN), mae (MIN)
creator.create("FitnessMulti", base.Fitness, weights=(1.0, -1.0, -1.0)) 

# A list-type individual with a fitness attribute
creator.create("Individual", list, fitness=creator.FitnessMulti)

# Instantiate a Toolbox to register all the evolutionary operators.
toolbox = base.Toolbox()


NDIM = 10
NOBJ = 3
p = 12
toolbox.pop_size = 30
toolbox.max_gen = 500
CXPB = 1.0
MUTPB = 1.0
ref_points = tools.uniform_reference_points(NOBJ, p)


""" Registering operators""" 
toolbox.register("generator", generator)

# Structure initializers
# define 'individual' to be a single individual taking up the values generated by the toolbox.generator. So we don't 
# need to repeat the toolbox.generator function. 
# This gives us flexibilty to define each parameter with its unique distribution, instead of keeping the distribution
# the same and applying it repeatedly across each parameter.

toolbox.register("individual", tools.initIterate, creator.Individual, 
    toolbox.generator)

# define the population to be a list of individuals # We don't define n here, but in the main body to give flexibility to num individuals.
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Operator registration
#----------
# register the goal / fitness function
toolbox.register("evaluate", evaluation_colored_icp, source_processed = source_processed, 
                 target_processed = target_processed, source_fpfh = source_fpfh, target_fpfh = target_fpfh,
                source_color = source_color, target_color = target_color)

# register the crossover operator
toolbox.register("mate", tools.cxTwoPoint)

# register a mutation operator 

toolbox.register("mutate", mutation, indpb = [0.2]*NDIM)

toolbox.register("select", tools.selNSGA3, ref_points = ref_points )



# DEAP main

In [12]:
def main(iter_ = 0, run_name = "./test"):
    random.seed(iter_ )
    
    # Initialize statistics object
     ## Creating logbook for recording statistics
        
    stats_o3d_fitness = tools.Statistics(lambda ind: ind.fitness.values[0])
    stats_o3d_rmse = tools.Statistics(lambda ind: ind.fitness.values[1])
    stats_o3d_mae = tools.Statistics(lambda ind: ind.fitness.values[2])
    mstats = tools.MultiStatistics(o3d_fitness=stats_o3d_fitness, 
                                   o3d_rmse =stats_o3d_rmse,
                                   o3d_mae = stats_o3d_mae)
    
    mstats.register("mean", lambda ind: round(sum(ind)/len(pop),3))
    mstats.register("max", lambda ind: round(np.max(ind),3))
    mstats.register("min", lambda ind: round(np.min(ind),3))
    
    logbook = tools.Logbook()
    logbook.header = ["gen", "evals", "o3d_fitness", "o3d_rmse", "o3d_mae"]
    logbook.chapters["o3d_fitness"].header = ["mean", "max"]
    logbook.chapters["o3d_rmse"].header = ["mean", "min"]
    logbook.chapters["o3d_mae"].header = ["mean", "min"]
    
    pop = toolbox.population(n= toolbox.pop_size)
        
    # Evaluate the individuals with an invalid fitness
    invalid_ind = [ind for ind in pop if not ind.fitness.valid]
    fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
        
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit[0:3]

    #Compile statistics about the population
    record = mstats.compile(pop)
    logbook.record(gen=0, evals=len(invalid_ind), **record)
    print("Initial logbook.stream", logbook.stream)
    
    for gen in range(1, toolbox.max_gen):

        
        #Apply crossover and mutation to generate new offsprings. Return a list of varied individuals that are independent of their parents.
        offspring = algorithms.varAnd(pop, toolbox, CXPB, MUTPB)
        fitnesses_offspring = [ind.fitness for ind in offspring if ind.fitness.valid]

        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        
        fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
        
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit[0:3]
            if fit[3].size: #If a correspondence map is present:
                corr_map_info = {"gen" : gen, "individual" : ind, "correspondence_set" : fit[3].tolist(), "fitness":ind.fitness.values[0],
                                 "inlier_rmse": ind.fitness.values[1], "mae": ind.fitness.values[2]}
                corr_map_results.append(corr_map_info)

        # Select the next generation population from parents and offspring
        pop = toolbox.select(pop + offspring, toolbox.pop_size)
      
        # Compile statistics about the new population
        record = mstats.compile(pop)
        logbook.record(gen=gen, evals=len(pop), **record)
        print(f"{gen} Statistics of next population: {logbook.stream}")
        
    corr_map_results_df = pd.DataFrame(corr_map_results)
    
    ## Saving best individuals and logbook
    newpath = run_name

    if not os.path.exists(newpath):
        os.makedirs(newpath)
           
    #np.save(file = f"{newpath}/result_ransac_df.csv", arr = best_k_ind_df)
    
    
    corr_map_results_df.to_csv(f'{newpath}/corrmapresults_coloredICP_run{iter_}.csv', index=False)
    
    results = {"gen" : logbook.select("gen"),
                "eval" : logbook.select("evals"),
                "o3d_fitness_mean" : logbook.chapters['o3d_fitness'].select("mean"),
                "o3d_fitness_max" : logbook.chapters['o3d_fitness'].select("max"),
                "o3d_rmse_mean" : logbook.chapters['o3d_rmse'].select("mean"),
                "o3d_rmse_min" : logbook.chapters['o3d_rmse'].select("min"),
                "o3d_mae_mean" : logbook.chapters['o3d_mae'].select("mean"),
                "o3d_mae_min" : logbook.chapters['o3d_mae'].select("min")}
    
    df_log = pd.DataFrame.from_dict(results) 
    df_log.to_csv(f'{newpath}/logbook_run{iter_}.csv', index=False) # Writing to a CSV file
    
merged_path_name = os.path.join(save_results_path, image_name, results_name)


# DEAP results

In [13]:
for i in range(num_deap_runs):
    run_name = f'{merged_path_name}_{i}_{date.today()}'
    main(iter_ = i, run_name = run_name)

Initial logbook.stream    	     	o3d_fitness	   o3d_rmse  	   o3d_mae   
   	     	-----------	-------------	-------------
gen	evals	mean	max   	mean 	min  	mean 	min  
0  	30   	0   	0     	10000	10000	10000	10000


C:\Users\User\anaconda3\envs\open3D\lib\site-packages\deap\tools\emo.py:606: RuntimeWarning: invalid value encountered in true_divide
  fn = (fitnesses - best_point) / (intercepts - best_point)


1 Statistics of next population: 1  	30   	0   	0     	10000	10000	10000	10000
2 Statistics of next population: 2  	30   	0   	0     	10000	10000	10000	10000
3 Statistics of next population: 3  	30   	0   	0     	10000	10000	10000	10000


KeyboardInterrupt: 

# Defunct code

In [ ]:
######################### 4) DEAP main #################################


def main(iter_ = 0, run_name = "./test"):
    random.seed(iter_ )
    
    # Initialize statistics object
     ## Creating logbook for recording statistics
        
    stats_o3d_fitness = tools.Statistics(lambda ind: ind.fitness.values[0])
    stats_o3d_rmse = tools.Statistics(lambda ind: ind.fitness.values[1])
    stats_o3d_mae = tools.Statistics(lambda ind: ind.fitness.values[2])
    mstats = tools.MultiStatistics(o3d_fitness=stats_o3d_fitness, 
                                   o3d_rmse =stats_o3d_rmse,
                                   o3d_mae = stats_o3d_mae)
    
    mstats.register("mean", lambda ind: round(sum(ind)/len(pop),3))
    mstats.register("max", lambda ind: round(np.max(ind),3))
    mstats.register("min", lambda ind: round(np.min(ind),3))
    
    logbook = tools.Logbook()
    logbook.header = ["gen", "evals", "o3d_fitness", "o3d_rmse", "o3d_mae"]
    logbook.chapters["o3d_fitness"].header = ["mean", "max"]
    logbook.chapters["o3d_rmse"].header = ["mean", "min"]
    logbook.chapters["o3d_mae"].header = ["mean", "min"]
    
    pop = toolbox.population(n= toolbox.pop_size)
        
    # Evaluate the individuals with an invalid fitness
    invalid_ind = [ind for ind in pop if not ind.fitness.valid]
    fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
        
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit[0:3]

    #Compile statistics about the population
    record = mstats.compile(pop)
    logbook.record(gen=0, evals=len(invalid_ind), **record)
    print("Initial logbook.stream", logbook.stream)
    
    # Best k individuals each gen (parameter values)
    #best_k_ind = []
    
    # Best k individuals each gen (coloredICPresults)
   # best_k_ind_results = []
    
    # Manual logging results with corr maps
    corr_map_results = []
    
    # Begin the generational process
    
    #print("Beginning generational process")
    for gen in range(1, toolbox.max_gen):
       # for ind in pop:
       #     print(f"{gen}---Parent: {ind}, Fitness: {ind.fitness.values}---")
        
        #Apply crossover and mutation to generate new offsprings. Return a list of varied individuals that are independent of their parents.
        offspring = algorithms.varAnd(pop, toolbox, CXPB, MUTPB)
        fitnesses_offspring = [ind.fitness for ind in offspring if ind.fitness.valid]
        #print(f"{gen}---Fitnesses of offsprings with valid fitnesses are {fitnesses_offspring}---")

        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        
        fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
        #count = 1
        
        for ind, fit in zip(invalid_ind, fitnesses):
            #print(f"ind: {ind}")
            ind.fitness.values = fit[0:3]
            if fit[3].size: #If a correspondence map is present:
                corr_map_info = {"gen" : gen, "individual" : ind, "correspondence_set" : fit[3].tolist(), "fitness":ind.fitness.values[0],
                                 "inlier_rmse": ind.fitness.values[1], "mae": ind.fitness.values[2]}
                corr_map_results.append(corr_map_info)
            #print(f"--- count: {count}; gen: {gen}; Invalid offspring: {ind}, Fitness: {ind.fitness.values}---")
            #count+=1

        # Select the next generation population from parents and offspring
        pop = toolbox.select(pop + offspring, toolbox.pop_size)
        
        """
        print(f"{gen}---List of selected individuals for next generation---")
        count = 1 
        for ind in pop:
            print(f"index:{count}; ind: {ind}") 
            count +=1 
        """
        # Compile statistics about the new population
        record = mstats.compile(pop)
        logbook.record(gen=gen, evals=len(pop), **record)
        print(f"{gen} Statistics of next population: {logbook.stream}")
        
        """
         # Updating best_k_ind dictionary
        best_ind = tools.selBest(pop, best_k)
        fitnesses = [list(ind.fitness.values) for ind in best_ind]
        results = toolbox.map(toolbox.results, best_ind)
            
        mapped_ind = list(zip(best_ind, fitnesses))
        mapped_results = list(zip(best_ind,results))
        
        for ind, fitness in mapped_ind:
            print(f"individual added to dict: {ind}, individual's fitness {fitness}")
            temp = {"gen":gen , "individual": ind, "o3d_fitness" : fitness[0], 
                              "o3d_rmse" : fitness[1], "o3d_mae" : fitness[2]}
            best_k_ind.append(temp)
            
        for ind, result in mapped_results:
            temp2 = {"gen":gen , "individual": ind, "ransac_transformation": result[0], "coloredICP_transformation" : result[1], 
                     "correspondence_set" : result[2].tolist(), "fitness" : result[3],
                     "inlier_rmse" : result[4], "mae": result[5]}
            best_k_ind_results.append(temp2)
        """
        
        #print(f"Current results: {corr_map_results}")

    print(f"--End of (successful) evolution --")
    
    ## Fittest individuals from the population
    """
    best_k_ind_df = pd.DataFrame(best_k_ind)      
    print(best_k_ind_df)
    
    best_k_ind_results = pd.DataFrame(best_k_ind_results)  
    print(best_k_ind_results)
    """
    
    corr_map_results_df = pd.DataFrame(corr_map_results)
    
    ## Saving best individuals and logbook
    newpath = run_name

    if not os.path.exists(newpath):
        os.makedirs(newpath)
           
    #np.save(file = f"{newpath}/result_ransac_df.csv", arr = best_k_ind_df)
    
    """
    best_k_ind_df.to_csv(f'{newpath}/indiv_coloredICP_run{iter_}.csv', index=False) 
    best_k_ind_results.to_csv(f'{newpath}/regisresults_coloredICP_run{iter_}.csv', index=False) 
    """
    
    corr_map_results_df.to_csv(f'{newpath}/corrmapresults_coloredICP_run{iter_}.csv', index=False)
    
    results = {"gen" : logbook.select("gen"),
                "eval" : logbook.select("evals"),
                "o3d_fitness_mean" : logbook.chapters['o3d_fitness'].select("mean"),
                "o3d_fitness_max" : logbook.chapters['o3d_fitness'].select("max"),
                "o3d_rmse_mean" : logbook.chapters['o3d_rmse'].select("mean"),
                "o3d_rmse_min" : logbook.chapters['o3d_rmse'].select("min"),
                "o3d_mae_mean" : logbook.chapters['o3d_mae'].select("mean"),
                "o3d_mae_min" : logbook.chapters['o3d_mae'].select("min")}
    
    df_log = pd.DataFrame.from_dict(results) 
    df_log.to_csv(f'{newpath}/logbook_run{iter_}.csv', index=False) # Writing to a CSV file
    
merged_path_name = os.path.join(save_results_path, image_name, results_name)
